In [7]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.3 MB/s eta 0:00:00


In [2]:
import re
import nltk

# Use to handle punctuation
nltk.download('punkt')

import numpy as np
from nltk.tokenize import word_tokenize
import emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Data Preparation

### Cleaning and tokenization

In [3]:
# Define corpus
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!"'

In [4]:
# Print original corpus
print(f"Corpus : {corpus}")

# Do the substitution
data = re.sub(r'[,!?;-]', '.', corpus)

# Print cleaned corpus
print(f"After cleaning punctuation : {data}")

Corpus : Who ❤️ "word embeddings" in 2020? I do!!!"
After cleaning punctuation : Who ❤️ "word embeddings" in 2020. I do..."


In [5]:
# Print the cleaned corpus
print(f"Initial string : {data}")

# Tokenize the cleaned corpus
data = nltk.word_tokenize(data)

# Print the tokenized version of the corpus
print(f"After tokenization : {data}")

Initial string : Who ❤️ "word embeddings" in 2020. I do..."
After tokenization : ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...', "''"]


In [13]:
# Print the tokenized version of the corpus
print(f"Initial list of tokens : {data}")

# Filter tokenized corpus using list comprehension
data  = [
    ch.lower() for ch in data
    if ch.isalpha()
    or ch == '.'
    or bool(emoji.emoji_list(ch))
]

# Print the tokenized and filtered version of the corpus
print(f"After cleaning : {data}")

Initial list of tokens : ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '...', "''"]
After cleaning : ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do']


In [12]:
def tokenize(corpus):
  data = re.sub(r'[,!?;-]+', '.', corpus)
  data = nltk.word_tokenize(data)
  data = [
      ch.lower() for ch in data
      if ch.isalpha()
      or ch == '.'
      or bool(emoji.emoji_list(ch))
  ]
  return data

In [ ]:
# Define new corpus
corpus = "I am happy because I am learning"

# Print new corpus
print(f"Corpus : {corpus}")

# Save tokenized version of corpus into 'words' variable
words = tokenize(corpus)

# Print the tokenzied version of the corpus